# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Описание даннных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

# Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к базе данных

In [ ]:
import pandas as pd
from sqlalchemy import create_engine 

In [ ]:
db_config = {'user': 'praktikum_student',
 'pwd': 'Sdf4$2;d-d30pp',
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432,
 'db': 'data-analyst-final-project-db'}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [ ]:
def select(sql):
  return pd.read_sql(sql,con=engine)

## Вывод первых строк таблиц

In [ ]:
books_sql = '''
SELECT *
FROM books
'''
books = select(books_sql)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [ ]:
authors_sql = '''
SELECT *
FROM authors
'''
authors = select(authors_sql)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [ ]:
publishers_sql = '''
SELECT *
FROM publishers
'''
publishers = select(publishers_sql)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [ ]:
ratings_sql = '''
SELECT *
FROM ratings
'''
ratings = select(ratings_sql)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [ ]:
reviews_sql = '''
SELECT *
FROM reviews
'''
reviews = select(reviews_sql)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Посчитаем, сколько книг вышло после 1 января 2000 года

In [ ]:
query = '''
SELECT COUNT(DISTINCT book_id) AS books_count
FROM books
WHERE publication_date >= '2000-01-01'
'''
select(query)

,books_count
0,819


819 книг было выпущено после 1 января 2000 года.

## Для каждой книги посчитаем количество обзоров и среднюю оценку

In [ ]:
query = '''
SELECT
 books.book_id AS id,
 books.title AS book_title,
 AVG(ratings.rating) AS AVG_rating,
 COUNT(DISTINCT reviews.review_id) AS number_of_reviews
FROM books
 INNER JOIN reviews ON reviews.book_id = books.book_id
 INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
 id,
 book_title
ORDER BY
 number_of_reviews DESC
LIMIT 10
'''
select(query)

,id,book_title,avg_rating,number_of_reviews
0,948,Twilight (Twilight #1),3.662500,7
1,696,The Da Vinci Code (Robert Langdon #2),3.830508,6
2,207,Eat Pray Love,3.395833,6
3,627,The Alchemist,3.789474,6
4,673,The Catcher in the Rye,3.825581,6
5,695,The Curious Incident of the Dog in the Night-Time,4.081081,6
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
7,299,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
8,497,Outlander (Outlander #1),4.125000,6
9,656,The Book Thief,4.264151,6


Вывели топ-10 книг по количеству обзоров. На первом месте книга "Twilight (Twilight #1)" со средним рейтингом 3.6. В топе нет книг с рейтингом 5.

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры

In [ ]:
query = '''
SELECT
 publishers.publisher_id AS id,
 publishers.publisher AS publisher,
 COUNT(DISTINCT books.book_id) AS number_of_books
FROM publishers
 LEFT JOIN books ON books.publisher_id = publishers.publisher_id
WHERE 
 books.num_pages > 50
GROUP BY
 id,
 publisher
ORDER BY
 number_of_books DESC
LIMIT 10
'''
select(query)

,id,publisher,number_of_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19
5,35,Bantam,19
6,45,Berkley,17
7,46,Berkley Books,14
8,284,St. Martin's Press,14
9,83,Delta,13


Издательство, выпустившее наибольшее количество книг, - издательство "Penguin Books".

## Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками

In [ ]:
query = '''
SELECT
   authors.author_id AS id,
   authors.author AS author,
   AVG(ratings.rating) AS AVG_books_rating
FROM
   authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
   id,
   author
HAVING 
   COUNT(ratings.rating) > 50
ORDER BY
 AVG_books_rating DESC
LIMIT 10
'''

select(query)

,id,author,avg_books_rating
0,236,J.K. Rowling/Mary GrandPré,4.288462
1,3,Agatha Christie,4.283019
2,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
3,240,J.R.R. Tolkien,4.240964
4,499,Roald Dahl/Quentin Blake,4.209677
5,376,Louisa May Alcott,4.203704
6,498,Rick Riordan,4.130952
7,39,Arthur Golden,4.107143
8,542,Stephen King,4.009434
9,302,John Grisham,3.971429


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [ ]:
query = '''
SELECT
 AVG (number_of_reviews) AS AVG_count
 FROM
 (
   SELECT
    COUNT (review_id) AS number_of_reviews
   FROM
    reviews
   INNER JOIN
   (SELECT
     username,
     COUNT (rating_id) AS number_of_ratings
    FROM
     ratings
    GROUP BY
     username
    HAVING COUNT (rating_id) > 50 ) AS ratings_users ON ratings_users.username = reviews.username
    GROUP BY
    reviews.username
 ) AS number_of_reviews
'''
select(query)

,avg_count
0,24.333333


Среднее количество обзоров от пользователей, поставивших более 50 оценок, - 24.

## Общий вывод

- После 1 января 2000 года было выпущено 819 книг.
- В топе- 10 книг по количеству обзоров на первом месте книга "Twilight (Twilight #1)" со средним рейтингом 3.6.
- Издательство, выпустившее наибольшее количество книг, - издательство "Penguin Books".
- Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.
- Среднее количество обзоров от пользователей, поставивших более 50 оценок, - 24.